## 01 – Tools and Tool Calling (Extending Models)

**Key Concept**: Base LLMs can only generate text. Tools let them perform actions: call APIs, run calculations, fetch data, etc.

**What this covers:**
1. What tools are and why they matter
2. Creating tools from Python functions
3. Binding tools to models
4. Manual tool-calling loop (foundation for agents)
5. Building a reusable toolset

In [1]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain_groq import ChatGroq

# For type hints and structured inputs
from typing import Annotated
from pydantic import BaseModel, Field

# Standard library
import os
from datetime import datetime
import json

/Users/yashsarode/Downloads/Personal Projects/Python/LangGraph-personal/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
llm = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct", 
    temperature=0,  # Deterministic for consistent tool usage
)

### What is a Tool?

A **tool** in LangChain is:
- A Python function with metadata (name, description, schema)
- Something the LLM can "call" when it needs external capabilities
- Executed in your Python runtime, not by the model

**Why tools matter:**
- LLMs can't do math reliably → Give them a calculator tool
- LLMs can't access real-time data → Give them API tools
- LLMs can't execute code → Give them interpreter tools

The `@tool` decorator automatically converts functions into LangChain tools.

In [5]:
@tool
def calculator(
    operation: Annotated[str, "The math operation: 'add', 'subtract', 'multiply', 'divide'"],
    a: Annotated[float, "First number"],
    b: Annotated[float, "Second number"]
) -> str:
    """Perform basic arithmetic operations. Use this when you need to calculate exact numerical results."""
    
    operations = {
        "add": a + b,
        "subtract": a - b,
        "multiply": a * b,
        "divide": a / b if b != 0 else "Error: Division by zero"
    }
    
    result = operations.get(operation.lower(), "Error: Invalid operation")
    return f"Result: {result}"

# what the tool looks like to the model
print(f"Tool name: {calculator.name}")
print(f"Tool description: {calculator.description}")
print(f"\nTool schema (what the model sees):")
print(json.dumps(calculator.args_schema.schema(), indent=2))

Tool name: calculator
Tool description: Perform basic arithmetic operations. Use this when you need to calculate exact numerical results.

Tool schema (what the model sees):
{
  "description": "Perform basic arithmetic operations. Use this when you need to calculate exact numerical results.",
  "properties": {
    "operation": {
      "description": "The math operation: 'add', 'subtract', 'multiply', 'divide'",
      "title": "Operation",
      "type": "string"
    },
    "a": {
      "description": "First number",
      "title": "A",
      "type": "number"
    },
    "b": {
      "description": "Second number",
      "title": "B",
      "type": "number"
    }
  },
  "required": [
    "operation",
    "a",
    "b"
  ],
  "title": "calculator",
  "type": "object"
}


/var/folders/_v/fs86q2353gvdsjh19x_1fpdm0000gn/T/ipykernel_93074/3076482416.py:23: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(json.dumps(calculator.args_schema.schema(), indent=2))


### Tool Design Best Practices

**1. Clear descriptions**: The LLM reads your docstring to decide when to use the tool  
**2. Typed parameters**: Use type hints so the model knows what inputs are valid  
**3. Return strings**: Models work best with string responses  
**4. Keep them focused**: One tool = one clear responsibility  

The `Annotated` type hints provide inline documentation that helps the model understand parameters.
`Annotated[Type, metadata]` = "This is of Type, AND here's extra information about it

Without `Annotated`, the LLM only sees:
```json
{
  "operation": "string",
  "a": "number",
  "b": "number"
}
```

With Annotated, the LLM sees:
```json
{
  "operation": {
    "type": "string",
    "description": "The math operation: 'add', 'subtract', 'multiply', 'divide'"
  },
  "a": {
    "type": "number", 
    "description": "First number"
  },
  "b": {
    "type": "number",
    "description": "Second number"
  }
}
```
Structured Tools with Pydantic can also be used instead of Annotated

```python
from pydantic import BaseModel, Field
from langchain_core.tools import tool

class CalculatorInput(BaseModel):
    """Input schema for calculator tool."""
    operation: str = Field(
        description="The math operation: 'add', 'subtract', 'multiply', 'divide'"
    )
    a: float = Field(description="First number")
    b: float = Field(description="Second number")

@tool(args_schema=CalculatorInput)
def calculator(operation: str, a: float, b: float) -> str:
    """Perform basic arithmetic operations."""
    # Implementation here
    pass
```

In [6]:
from pydantic import BaseModel, Field
from typing import Literal

class WeatherInput(BaseModel):
    """Input for weather queries."""
    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include 5-day forecast"
    )

@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

# what the tool looks like to the model
print(f"Tool name: {get_weather.name}")
print(f"Tool description: {get_weather.description}")
print(f"\nTool schema (what the model sees):")
print(json.dumps(get_weather.args_schema.schema(), indent=2))

Tool name: get_weather
Tool description: Get current weather and optional forecast.

Tool schema (what the model sees):
{
  "description": "Input for weather queries.",
  "properties": {
    "location": {
      "description": "City name or coordinates",
      "title": "Location",
      "type": "string"
    },
    "units": {
      "default": "celsius",
      "description": "Temperature unit preference",
      "enum": [
        "celsius",
        "fahrenheit"
      ],
      "title": "Units",
      "type": "string"
    },
    "include_forecast": {
      "default": false,
      "description": "Include 5-day forecast",
      "title": "Include Forecast",
      "type": "boolean"
    }
  },
  "required": [
    "location"
  ],
  "title": "WeatherInput",
  "type": "object"
}


/var/folders/_v/fs86q2353gvdsjh19x_1fpdm0000gn/T/ipykernel_93074/3131039715.py:29: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(json.dumps(get_weather.args_schema.schema(), indent=2))


### Binding Tools to Models

Now comes the magic: we tell the model about our tools using `bind_tools()`.

**What happens behind the scenes:**
1. LangChain converts tool schemas to the format your model expects (OpenAI, Anthropic, etc.)
2. The model can now "see" available tools in its context
3. When needed, the model outputs a **tool call** instead of text

**Important**: Not all models support tool calling. Check your model's capabilities.

In [10]:
# Create our toolset
tools = [calculator, get_weather]

# Bind tools to the model
# This doesn't execute anything yet - just tells the model tools exist
llm_with_tools = llm.bind_tools(tools)

print(f"✓ Bound {len(tools)} tools to model")
print(f"Tools available: {[t.name for t in tools]}")

✓ Bound 2 tools to model
Tools available: ['calculator', 'get_weather']


### How Tool Calling Works

**User asks a question** → Model decides:
- **Option A**: Answer directly (no tools needed)
- **Option B**: Call one or more tools, then synthesize final answer

When the model chooses Option B, it returns an `AIMessage` with `tool_calls` instead of content.

In [11]:
# Question that doesn't need tools
response = llm_with_tools.invoke([
    HumanMessage(content="What is the capital of France?")
])

print("Response type:", type(response).__name__)
print("Content:", response.content)
print("Tool calls:", response.tool_calls)  # Should be empty

Response type: AIMessage
Content: The capital of France is Paris.
Tool calls: []


In [12]:
# Question requiring calculation
response = llm_with_tools.invoke([
    HumanMessage(content="What is 847 multiplied by 923?")
])

print("Response type:", type(response).__name__)
print("Content:", response.content)
print("\nTool calls made:")
print(json.dumps(response.tool_calls, indent=2))

Response type: AIMessage
Content: 

Tool calls made:
[
  {
    "name": "calculator",
    "args": {
      "a": 847,
      "b": 923,
      "operation": "multiply"
    },
    "id": "b9nywn478",
    "type": "tool_call"
  }
]


### Manual Tool Execution Loop

When the model returns a tool call, **we** need to:
1. Execute the Python function
2. Send the result back to the model
3. Let the model generate the final answer

This is what agents automate. But understanding this flow is crucial.

**The loop:**
```
User Query → Model → Tool Call? → Execute Tool → Model → Final Answer
```

In [13]:
def run_tool_calling_flow(user_query: str):
    """
    Manual implementation of what agents do automatically.
    This helps understand the mechanics before we abstract it away.
    """
    
    print(f"\n{'='*60}")
    print(f"USER: {user_query}")
    print('='*60)
    
    # Step 1: Send query to model
    messages = [HumanMessage(content=user_query)]
    response = llm_with_tools.invoke(messages)
    
    # Step 2: Check if model wants to call tools
    if not response.tool_calls:
        print(f"\nMODEL (Direct answer): {response.content}")
        return response.content
    
    # Step 3: Model called tools - execute them
    print(f"\nMODEL: I need to call {len(response.tool_calls)} tool(s)...")
    messages.append(response)  # Add model's tool call to history
    
    # Create a mapping of tool names to actual functions
    tool_map = {t.name: t for t in tools}
    
    for tool_call in response.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]
        
        print(f"\n  → Calling: {tool_name}")
        print(f"    Arguments: {tool_args}")
        
        # Execute the tool
        selected_tool = tool_map[tool_name]
        tool_output = selected_tool.invoke(tool_args)
        
        print(f"    Result: {tool_output}")
        
        # Step 4: Add tool result to message history
        messages.append(ToolMessage(
            content=str(tool_output),
            tool_call_id=tool_id,
            name=tool_name
        ))
    
    # Step 5: Send everything back to model for final answer
    final_response = llm_with_tools.invoke(messages)
    print(f"\nMODEL (Final answer): {final_response.content}")
    
    return final_response.content


In [14]:
# Test 1: Simple calculation
run_tool_calling_flow("What is 15% of 2,450?")


USER: What is 15% of 2,450?

MODEL: I need to call 1 tool(s)...

  → Calling: calculator
    Arguments: {'a': 2450, 'b': 0.15, 'operation': 'multiply'}
    Result: Result: 367.5

MODEL (Final answer): The result of 15% of 2,450 is 367.5.


'The result of 15% of 2,450 is 367.5.'